In [1]:
from utils import *
from net import Net
from googlenet import GoogLeNet
from dogs_dataset import DogsDataset

import torch
from torch.utils.data import DataLoader
from sklearn.preprocessing import LabelEncoder

In [2]:
#Importation du dataset
train_path = "../dataset/train" 
test_path = "../dataset/test" 

train_dataset_X, train_dataset_Y = load_dataset(train_path)
test_dataset_X, test_dataset_Y = load_dataset(test_path)

debut chargement image
debut chargement image


In [4]:
encoder = LabelEncoder()

train_dataset_Y = encoder.fit_transform(train_dataset_Y)
test_dataset_Y = encoder.fit_transform(test_dataset_Y)

In [5]:
#Creation des dataloader de train et test
dataset_train = DogsDataset(train_dataset_X, train_dataset_Y)
dataset_test = DogsDataset(test_dataset_X, test_dataset_Y)

train_loader = DataLoader(dataset=dataset_train,
                          batch_size=50,
                          shuffle=True,
                          num_workers=0)

test_loader = DataLoader(dataset=dataset_test,
                          shuffle=False,
                          batch_size=1,
                          num_workers=0)

torch.Size([12000, 3, 224, 224])
torch.Size([12000])
torch.Size([8580, 3, 224, 224])
torch.Size([8580])


In [6]:
# Recuperation du type d'execution
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Definition des hyperparametres
n_epoch = 1
lr = 0.0005
weight_decay = 0.0005

model = GoogLeNet()

#Si on entraine sur gpu 
if device.type == 'cuda':
    model.to(device)

train_losses = train(n_epoch, model, train_loader, lr, weight_decay, device)    

Epoch :  1 	 loss : tensor(4.3183, device='cuda:0', grad_fn=<NllLossBackward>)


In [15]:
# Pour sauvegarder le modele
path = "../models/googlenet_" + str(n_epoch) + '_' + str(lr) + '_' + str(weight_decay) + '.pth'
torch.save(model.state_dict(), path)

'cuda'

In [6]:
# Pour charger une modele de type GoogLeNet
model = GoogLeNet()
path = '../models/googlenet_1000_l2.pth'
model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [9]:
# Si on test sur le gpu
if device.type == 'cuda':
    model.to(device)
accuracy = test(model, test_loader, device )